# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet' , 'stopwords', 'averaged_perceptron_tagger' ])
from sqlalchemy import create_engine
from nltk import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    stop_word=stopwords.words('english')
    text=re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',text)
    text=re.sub(r'[^0-9A-Za-z]',' ',text)
   
    tokens=word_tokenize(text)
    lemmatizer=WordNetLemmatizer()
    
   
    tokens_data=[]
    for i in tokens:
        if i not in stop_word:
            clean_token=lemmatizer.lemmatize(i).lower().strip()
            
            clean_token=lemmatizer.lemmatize(clean_token,pos='v')
            tokens_data.append(clean_token)
    return tokens_data

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                     ('multiclf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=42)

In [6]:
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pre=pipeline.predict(X_test)

In [8]:
Y_pre=pd.DataFrame(Y_pre)

In [9]:
def result(y_predict):
    for index, column in enumerate(Y_test):
        print('column: ',column)
        print(classification_report(Y_test.iloc[:,index],y_predict.iloc[:,index]))
        
result(Y_pre)

column:  related
             precision    recall  f1-score   support

          0       0.61      0.46      0.53      1563
          1       0.84      0.90      0.87      4944
          2       0.38      0.40      0.39        47

avg / total       0.78      0.79      0.78      6554

column:  request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5443
          1       0.79      0.41      0.54      1111

avg / total       0.87      0.88      0.87      6554

column:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

column:  aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3884
          1       0.74      0.60      0.66      2670

avg / total       0.75      0.75      0.74      6554

column:  medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5edc5a8620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multiclf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state

In [11]:
parameters = parameters = {'vect__ngram_range':[(1,2)],
              'vect__max_df':[.5],
              'vect__max_features':[None,5000]
             }

cv = GridSearchCV(pipeline,parameters,n_jobs=-1,verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train,Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2) 
[CV]  vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2), score=0.24824534635337198, total= 1.6min
[CV] vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2), score=0.2436679890143424, total= 1.6min
[CV] vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  vect__max_df=0.5, vect__max_features=None, vect__ngram_range=(1, 2), score=0.23481843149221848, total= 1.6min
[CV] vect__max_df=0.5, vect__max_features=5000, vect__ngram_range=(1, 2) 


...

[CV]  vect__max_df=0.5, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.2554165395178517, total=  40.5s
[CV] vect__max_df=0.5, vect__max_features=5000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  7.2min remaining:    0.0s


[CV]  vect__max_df=0.5, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.25282270369240156, total=  40.5s


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 2)], 'vect__max_df': [0.5], 'vect__max_features': [None, 5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [13]:
Y_tune_pre=cv.predict(X_test)
Y_tune_pre=pd.DataFrame(Y_tune_pre)
result(Y_tune_pre)

column:  related
             precision    recall  f1-score   support

          0       0.61      0.48      0.54      1563
          1       0.84      0.90      0.87      4944
          2       0.31      0.30      0.30        47

avg / total       0.79      0.80      0.79      6554

column:  request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.78      0.46      0.58      1111

avg / total       0.88      0.89      0.87      6554

column:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

column:  aid_related
             precision    recall  f1-score   support

          0       0.77      0.84      0.80      3884
          1       0.73      0.65      0.69      2670

avg / total       0.76      0.76      0.76      6554

column:  medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
cv.best_params_
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 2)], 'vect__max_df': [0.5], 'vect__max_features': [None, 5000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 23)
pipeline1.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [18]:
Y_pred_p2 = pipeline1.predict(X_test)
for i in range(Y.shape[1]):
    print ('Category: "', Y.columns[i], '"')
    print(classification_report(Y_test.iloc[:,1].values, Y_pred_p2[:,i]))    

Category: " related "
             precision    recall  f1-score   support

          0       0.92      0.06      0.12      5390
          1       0.18      0.98      0.31      1164
          2       0.00      0.00      0.00         0

avg / total       0.79      0.23      0.15      6554

Category: " request "
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5390
          1       0.76      0.50      0.60      1164

avg / total       0.87      0.88      0.87      6554

Category: " offer "
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      5390
          1       0.22      0.00      0.00      1164

avg / total       0.72      0.82      0.74      6554

Category: " aid_related "
             precision    recall  f1-score   support

          0       0.91      0.72      0.81      5390
          1       0.34      0.67      0.46      1164

avg / total       0.81      0.72      0.74      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
accuracy = (Y_pred_p2 == Y_test).mean().mean()
print('Accuracy of the model: ', accuracy)

Accuracy of the model:  0.946932322924


### 9. Export your model as a pickle file

In [20]:
pickle.dump(cv, open('new_model.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.